# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770813431738                   -0.53    9.0         
  2   -2.772140124642       -2.88       -1.31    1.0    171ms
  3   -2.772170138562       -4.52       -2.59    1.0    174ms
  4   -2.772170718923       -6.24       -3.90    2.0    238ms
  5   -2.772170722557       -8.44       -4.24    2.0    200ms
  6   -2.772170723008       -9.35       -5.56    1.0    200ms
  7   -2.772170723015      -11.15       -6.09    2.0    204ms
  8   -2.772170723015      -13.17       -6.84    1.0    207ms
  9   -2.772170723015   +    -Inf       -7.38    2.0    220ms
 10   -2.772170723015      -13.60       -7.99    1.0    215ms
 11   -2.772170723015   +  -14.03       -8.87    2.0    211ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770768317203                   -0.52    9.0         
  2   -2.7

1.7735579954884655

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770468951162                   -0.53    9.0         
  2   -2.772052784252       -2.80       -1.29    1.0    173ms
  3   -2.772082775252       -4.52       -2.77    1.0    245ms
  4   -2.772083415883       -6.19       -4.10    2.0    197ms
  5   -2.772083417794       -8.72       -4.80    2.0    201ms
  6   -2.772083417809      -10.83       -5.48    1.0    208ms
  7   -2.772083417811      -11.75       -6.32    2.0    206ms
  8   -2.772083417811      -14.05       -7.19    1.0    205ms
  9   -2.772083417811      -14.01       -7.85    2.0    210ms
 10   -2.772083417811   +  -14.31       -8.22    1.0    218ms

Polarizability via ForwardDiff:       1.772534989818863
Polarizability via finite difference: 1.7735579954884655
